In [1]:
import os
import sys

In [2]:
%pwd

'c:\\TextSummarizerProject\\reasearch'

In [10]:
os.chdir("../")

In [4]:
## UPDATE ENTITY
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [12]:
%pwd

'c:\\TextSummarizerProject'

In [13]:
## UPDATE CONFIGURATION MANAGER

from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories


In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

       
        self.artifacts_root = self.config.get("artifacts_root")

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.get("data_ingestion")  
        create_directories([config["root_dir"]])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config["root_dir"],
            source_URL=config["source_URL"],
            local_data_file=config["local_data_file"],
            unzip_dir=config["unzip_dir"]
        )

        return data_ingestion_config


In [15]:
## UPDATE COMPONENTS

import os
import urllib.request as request         #DOWNLOAD DATA
import zipfile                           #TO UNZIP DATA
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
##UPADATE THE PIPELINE

from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

try:
    config = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


##INITILIZE CONFIGURATIUON MANAGER
# TAKE DATA INGESTION CONFIG AND STORE IN DATA_INGETSION_CONFIG
#CALL DATAINGESTION COMPONENT AND PASS DATAINGESTIONCONFIG INSDIE IT
#then inititae two methods
    


[2023-08-29 09:32:01,680:INFO:common:yaml file: config\config.yaml loaded successfully]
[2023-08-29 09:32:01,683:INFO:common:yaml file: params.yaml loaded successfully]
[2023-08-29 09:32:01,688:INFO:common:created directory at: artifacts/data_ingestion]
[2023-08-29 09:32:09,839:INFO:1434958058:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FF14:298320:2C3E2:4BD1E:64ED6DB9
Accept-Ranges: bytes
Date: Tue, 29 Aug 2023 04:02:04 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830021-CCU
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1693281724.912972,VS0,VE774
Vary: Authorization,Accept-E